In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/PRJ-3563_archive.zip'

Archive:  /content/drive/MyDrive/PRJ-3563_archive.zip
 extracting: PRJ-3563/Ida-BD_description.pdf  
 extracting: PRJ-3563/images/AOI1-tile_10-6_post_disaster.png  
 extracting: PRJ-3563/images/AOI1-tile_7-6_post_disaster.png  
 extracting: PRJ-3563/images/AOI1-tile_1-3_post_disaster.png  
 extracting: PRJ-3563/images/AOI1-tile_4-5_post_disaster.png  
 extracting: PRJ-3563/images/AOI3-tile_6-3_pre_disaster.png  
 extracting: PRJ-3563/images/AOI1-tile_2-3_pre_disaster.png  
 extracting: PRJ-3563/images/AOI1-tile_4-3_post_disaster.png  
 extracting: PRJ-3563/images/AOI3-tile_4-6_pre_disaster.png  
 extracting: PRJ-3563/images/AOI3-tile_5-2_pre_disaster.png  
 extracting: PRJ-3563/images/AOI1-tile_6-3_post_disaster.png  
 extracting: PRJ-3563/images/AOI3-tile_6-2_post_disaster.png  
 extracting: PRJ-3563/images/AOI1-tile_11-3_post_disaster.png  
 extracting: PRJ-3563/images/AOI3-tile_1-0_post_disaster.png  
 extracting: PRJ-3563/images/AOI3-tile_6-5_pre_disaster.png  
 extracting: PRJ-356

In [ ]:
import os
import numpy as np
from PIL import Image

def assign_label(image_name):
    if "post_disaster" in image_name:
        return 1
    else:
        return 0

def create_dataset(images_folder):
    dataset = []
    for image_name in os.listdir(images_folder):
        if image_name.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(images_folder, image_name)
            label = assign_label(image_name)
            dataset.append((image_path, label))
    return dataset

def create_hyperspectral(dataset):
    images = []
    labels = []
    for image_path, label in dataset[:50]:
        img = Image.open(image_path)
        img_array = np.array(img)
        images.append(img_array)
        labels.append(label)
    return np.array(images), np.array(labels)

images_folder = "PRJ-3563/images"
dataset = create_dataset(images_folder)
X, y = create_hyperspectral(dataset)

# Convert RGB images to hyperspectral images
# def convert_to_hyperspectral(rgb_images):
#     spectral_bands = []
#     for img in rgb_images:
#         red = img[:, :, 0]
#         green = img[:, :, 1]
#         blue = img[:, :, 2]
#         spectral_bands.append([red, green, blue])

#     print(np.array(spectral_bands).shape)
#     return np.array(spectral_bands)

# X = convert_to_hyperspectral(X)

print("Hyperspectral images created successfully.")
print("Total hyperspectral images:", len(X))
print("Total labels:", len(y))


Hyperspectral images created successfully.
Total hyperspectral images: 50
Total labels: 50


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, 0.2)

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.layers import Input, Conv2D, Reshape, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

S = X.shape[0]
L = X.shape[1]
output_units = 1

input_layer = Input((L , L , 3))


conv_layer1 = Conv2D(filters=8, kernel_size=(3, 3), activation='relu' , padding = 'same')(input_layer)
# conv_layer2 = Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(conv_layer1)
# conv_layer3 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same')(conv_layer2)

# conv2d_shape = conv_layer3.shape
flatten_layer = Flatten()(conv_layer1)

dense_layer1 = Dense(units=64, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=32, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=output_units, activation='sigmoid')(dense_layer2)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1024, 1024, 3)]   0         
                                                                 
 conv2d_10 (Conv2D)          (None, 1024, 1024, 8)     224       
                                                                 
 flatten_7 (Flatten)         (None, 8388608)           0         
                                                                 
 dense_21 (Dense)            (None, 64)                536870976 
                                                                 
 dropout_14 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dropout_15 (Dropout)        (None, 32)                0   

In [ ]:
import keras
y = keras.utils.to_categorical(y)
y.shape

(50, 2)

In [ ]:
import tensorflow as tf
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# ytrain = keras.utils.to_categorical(ytrain)
# ytrain.shape

# ytest = keras.utils.to_categorical(ytest)
# y.shape

(50, 2)

In [ ]:
history = model.fit(x=np.array(Xtrain), y=ytrain, validation_data=(np.array(Xtest), ytest), batch_size=256, epochs=100, verbose=1)


Epoch 1/100
1/1 [==============================] - 62s 62s/step - loss: 1.4438 - accuracy: 0.4500 - val_loss: 24.0666 - val_accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 55s 55s/step - loss: 57.1534 - accuracy: 0.6750 - val_loss: 5.6427 - val_accuracy: 0.7000
Epoch 3/100
1/1 [==============================] - 55s 55s/step - loss: 57.1743 - accuracy: 0.7250 - val_loss: 39.5322 - val_accuracy: 0.6000
Epoch 4/100
1/1 [==============================] - 55s 55s/step - loss: 73.5232 - accuracy: 0.5500 - val_loss: 62.1171 - val_accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 56s 56s/step - loss: 79.4427 - accuracy: 0.6500 - val_loss: 11.1722 - val_accuracy: 0.7000
Epoch 6/100
1/1 [==============================] - 55s 55s/step - loss: 65.0258 - accuracy: 0.6500 - val_loss: 5.4862 - val_accuracy: 0.8000
Epoch 7/100
1/1 [==============================] - 55s 55s/step - loss: 42.2912 - accuracy: 0.7750 - val_loss: 24.4856 - val_accuracy: 0.7000
Epoch 8/1

KeyboardInterrupt: 

In [ ]:
import pickle

# Save the model using pickle
with open('model2.pkl', 'wb') as f:
    pickle.dump(model, f)


In [ ]:
import shutil

source_file_path = 'model.pkl'

destination_folder_path = '/content/drive/My Drive/Colab Notebooks/'

shutil.copy(source_file_path, destination_folder_path)


'/content/drive/My Drive/Colab Notebooks/model.pkl'

In [ ]:
import shutil

source_file_path = 'model2.pkl'

destination_folder_path = '/content/drive/My Drive/Colab Notebooks/'

shutil.copy(source_file_path, destination_folder_path)


'/content/drive/My Drive/Colab Notebooks/model2.pkl'

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

S = X.shape[0]
L = X.shape[1]
output_units = 1

# Load ResNet50 without the top layer
resnet = ResNet50(include_top=False, input_shape=(L, L, 3))

# Freeze the pre-trained layers
for layer in resnet.layers:
    layer.trainable = False

# Flatten the output of ResNet
flatten = Flatten()(resnet.output)

# Add Dense layers for custom classification
dense1 = Dense(units=64, activation='relu')(flatten)
dropout1 = Dropout(0.4)(dense1)
dense2 = Dense(units=32, activation='relu')(dropout1)
dropout2 = Dropout(0.4)(dense2)
output_layer = Dense(units=output_units, activation='sigmoid')(dropout2)

# Define the model
model = Model(inputs=resnet.input, outputs=output_layer)
model.summary()


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_12 (InputLayer)       [(None, 1024, 1024, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 1030, 1030, 3)        0         ['input_12[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 512, 512, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 512, 512, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        